In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/Year 4/Thesis/Notebooks/Source Code/')

In [ ]:
!ls

'All Apps Review-Nandan & Jing & Tung'	'Crawl Valuable Data.ipynb'
 app_ids.csv				 reviews-com.mservice.momotransfer_06-04-2024_032238.csv


In [ ]:
!pip install google_play_scraper pymongo tzlocal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# for scraping app info and reviews from Google Play
from google_play_scraper import app, Sort, reviews

# for pretty printing data structures
from pprint import pprint

# for storing in MongoDB
import pymongo
from pymongo import MongoClient

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

In [ ]:
# Set up Mongo client
client = MongoClient(host='localhost', port=27017)

# Database for project
app_proj_db = client['app_proj_db']

# Set up new collection within project db for app info
info_collection = app_proj_db['info_collection']

# Set up new collection within project db for app reviews
review_collection = app_proj_db['review_collection']

In [ ]:
# Read in file containing app names and IDs
app_df = pd.read_csv('app_ids.csv')
app_df.head()

,app_name,android_appID
0,Momo,com.mservice.momotransfer
1,ShopeePay,com.beeasy.toppay
2,VNPay,vnpay.smartacccount
3,ZaloPay,vn.com.vng.zalopay
4,ViettelPay,com.bplus.vtpay


In [ ]:
# Get list of app names and app IDs
app_names = list(app_df['app_name'])
app_ids = list(app_df['android_appID'])

In [ ]:
# Loop through app IDs to get app info
app_info = []
for i in app_ids:
    info = app(i)
    del info['comments']
    app_info.append(info)

# Pretty print the data for the first app
pprint(app_info[0])

{'adSupported': False,
 'appId': 'com.mservice.momotransfer',
 'categories': [{'id': 'FINANCE', 'name': 'Finance'}],
 'containsAds': False,
 'contentRating': 'Everyone',
 'contentRatingDescription': None,
 'currency': 'USD',
 'description': 'MOMO - PAYMENT & FINANCE SUPER APP - ALL IN ONE\r\n'
                '\r\n'
                'MoMo is proud to be a Payment - Finance Super Application '
                'that meets all the life needs of Vietnamese people. All the '
                'services you need - are available on MoMo, including: Money '
                'transfer, Payment, Accumulate profits with Fortune Bag and '
                'features such as Top up your phone, Buy 3G/4G data, Pay all '
                'bills , Buy movie tickets, Book plane tickets, train '
                'tickets... In addition, you can also scan QR codes to pay for '
                'online services Lazada, Tiki... and directly at supermarkets, '
                'stores, restaurants …\r\n'
            

In [ ]:
app_id = ['com.mservice.momotransfer', 'com.beeasy.toppay', 'vnpay.smartacccount', 'vn.com.vng.zalopay', 'com.bplus.vtpay', 'com.paypal.android.p2pmobile']

  0%|          | 0/25000 [00:00<?, ?it/s]


(0, 0)

In [ ]:
import json
from time import sleep
from typing import List, Optional, Tuple

In [ ]:
from google_play_scraper import Sort
from google_play_scraper.constants.element import ElementSpecs
from google_play_scraper.constants.regex import Regex
from google_play_scraper.constants.request import Formats
from google_play_scraper.utils.request import post

import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

In [ ]:
app_id = 'vnpay.smartacccount'

MAX_COUNT_EACH_FETCH = 199


class _ContinuationToken:
    __slots__ = (
        "token",
        "lang",
        "country",
        "sort",
        "count",
        "filter_score_with",
        "filter_device_with",
    )

    def __init__(
        self, token, lang, country, sort, count, filter_score_with, filter_device_with
    ):
        self.token = token
        self.lang = lang
        self.country = country
        self.sort = sort
        self.count = count
        self.filter_score_with = filter_score_with
        self.filter_device_with = filter_device_with


def _fetch_review_items(
    url: str,
    app_id: str,
    sort: int,
    count: int,
    filter_score_with: Optional[int],
    filter_device_with: Optional[int],
    pagination_token: Optional[str],
):
    dom = post(
        url,
        Formats.Reviews.build_body(
            app_id,
            sort,
            count,
            "null" if filter_score_with is None else filter_score_with,
            "null" if filter_device_with is None else filter_device_with,
            pagination_token,
        ),
        {"content-type": "application/x-www-form-urlencoded"},
    )
    match = json.loads(Regex.REVIEWS.findall(dom)[0])

    return json.loads(match[0][2])[0], json.loads(match[0][2])[-2][-1]


def reviews(
    app_id: str,
    lang: str = "vi",
    country: str = "vn",
    sort: Sort = Sort.MOST_RELEVANT,
    count: int = 100,
    filter_score_with: int = 3,
    filter_device_with: int = None,
    continuation_token: _ContinuationToken = None,
) -> Tuple[List[dict], _ContinuationToken]:
    sort = sort.value

    if continuation_token is not None:
        token = continuation_token.token

        if token is None:
            return (
                [],
                continuation_token,
            )

        lang = continuation_token.lang
        country = continuation_token.country
        sort = continuation_token.sort
        count = continuation_token.count
        filter_score_with = continuation_token.filter_score_with
        filter_device_with = continuation_token.filter_device_with
    else:
        token = None

    url = Formats.Reviews.build(lang=lang, country=country)

    _fetch_count = count

    result = []

    while True:
        if _fetch_count == 0:
            break

        if _fetch_count > MAX_COUNT_EACH_FETCH:
            _fetch_count = MAX_COUNT_EACH_FETCH

        try:
            review_items, token = _fetch_review_items(
                url,
                app_id,
                sort,
                _fetch_count,
                filter_score_with,
                filter_device_with,
                token,
            )
        except (TypeError, IndexError):
            #funnan MOD start
            token = continuation_token.token
            continue
            #MOD end

        for review in review_items:
            result.append(
                {
                    k: spec.extract_content(review)
                    for k, spec in ElementSpecs.Review.items()
                }
            )

        _fetch_count = count - len(result)

        if isinstance(token, list):
            token = None
            break

    return (
        result,
        _ContinuationToken(
            token, lang, country, sort, count, filter_score_with, filter_device_with
        ),
    )


def reviews_all(app_id: str, sleep_milliseconds: int = 0, **kwargs) -> list:
    kwargs.pop("count", None)
    kwargs.pop("continuation_token", None)

    continuation_token = None

    result = []

    while True:
        _result, continuation_token = reviews(
            app_id,
            count=MAX_COUNT_EACH_FETCH,
            continuation_token=continuation_token,
            **kwargs
        )

        result += _result

        if continuation_token.token is None:
            break

        if sleep_milliseconds:
            sleep(sleep_milliseconds / 1000)

    return result

In [ ]:
result = []
continuation_token = None
reviews_count = 199

with tqdm(total=reviews_count, position=0, leave=True) as pbar:
    while len(result) < reviews_count:
        new_result, continuation_token = reviews(
            app_id,
            continuation_token=continuation_token,
            lang='vi',
            country='vn',
            sort=Sort.NEWEST,
            filter_score_with=3,
            count=199
        )
        if not new_result:
            break
        result.extend(new_result)
        pbar.update(len(new_result))

100%|██████████| 199/199 [00:00<00:00, 1311.95it/s]


In [ ]:
neu_vnpay = pd.DataFrame(result)
neu_vnpay['app_name'] = app_id
neu_vnpay.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,app_name
0,763f4522-f5a5-46df-8b8d-d8e166ac71c4,Phương Thanh Nguyễn,https://play-lh.googleusercontent.com/a-/ALV-U...,Tại sao mình vào mấy lần nhưng toàn báo là kết...,3,2,2.25.1,2024-06-01 14:23:51,Ví VNPAY rất tiếc về vấn đề Quý khách đang gặp...,2024-06-02 07:41:55,2.25.1,vnpay.smartacccount
1,6bb9b631-423f-4bf3-8dfa-6f1de3152b83,Vân Anh Hoàng,https://play-lh.googleusercontent.com/a-/ALV-U...,Không load vào app được,3,0,2.23.1,2024-05-31 07:05:23,Ví VNPAY rất tiếc về vấn đề Quý khách đang gặp...,2024-05-31 13:35:08,2.23.1,vnpay.smartacccount
2,1d5c0172-26be-4737-ab6c-e67eec6cd2aa,Tg Dj,https://play-lh.googleusercontent.com/a/ACg8oc...,Định danh mà mãi chả được duyệt,3,0,2.24.6,2024-05-27 23:19:52,"Xin chào quý khách, Ví VNPAY rất tiếc về vấn đ...",2024-05-28 08:48:50,2.24.6,vnpay.smartacccount
3,643e0e2f-b720-4352-9c18-7d519dea7b21,Thương DoanHoai,https://play-lh.googleusercontent.com/a/ACg8oc...,Sao thanh toán bách hóa xanh không thấy và tha...,3,0,2.24.6,2024-05-27 09:24:19,Ví VNPAY rất tiếc về vấn đề Quý khách đang gặp...,2024-05-28 08:49:52,2.24.6,vnpay.smartacccount
4,b9868033-383f-45b0-8214-a8d32348cb75,Hiếu Phạm,https://play-lh.googleusercontent.com/a-/ALV-U...,Sao giờ không thể mua vé tàu hỏa trên app. Mở ...,3,3,2.24.6,2024-05-23 13:13:26,None,NaT,2.24.6,vnpay.smartacccount


In [ ]:
neu_vnpay.to_csv('data/neu_vnpay.csv', index = False)

In [ ]:
# Merge multiples dataset
reviews = pd.concat(map(pd.read_csv, ['data/momo.csv', 'data/shopeepay.csv', 'data/zalopay.csv', 'data/vnpay.csv', 'data/viettelpay.csv']), ignore_index=True)
reviews.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/vtpay.csv'

In [ ]:
reviews.shape

(50745, 12)

In [ ]:
reviews['app_name'].value_counts()

app_name
com.mservice.momotransfer    10149
com.beeasy.toppay            10149
vn.com.vng.zalopay           10149
vnpay.smartacccount          10149
com.bplus.vtpay              10149
Name: count, dtype: int64

In [ ]:
reviews.to_csv('data/e-wallet.csv', index = False)